In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import fig_settings as fs

In [ ]:
colors = ["#2d5d83", "#4f9bd9", "#077187", "#3dfaff", "#8b888e"]
fs.set_fonts()

In [ ]:
with open("data/starterpack-stats.json") as f:
    data = json.load(f)

component_sizes, component_numbers = data["components"]
edge_sizes = np.array(data["edge-sizes"])
degrees = np.array(data["degrees"])

creation_dates = [datetime.strptime(date, "%Y-%m-%d") for date in data["date-created"]]
n_starterpacks_created = np.array(data["number-created"])
num_nodes = data["num-nodes"]
num_edges = data["num-edges"]

min_edge_size = data["min-edge-size"]
max_edge_size = data["max-edge-size"]
mean_edge_size = data["mean-edge-size"]
median_edge_size = data["median-edge-size"]
mode_edge_size = data["mode-edge-size"]

min_degree = data["min-degree"]
max_degree = data["max-degree"]
mean_degree = data["mean-degree"]
median_degree = data["median-degree"]
mode_degree = data["mode-degree"]

account_age_at_creation = data["account-age-at-creation"]

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(131)
ax = plt.gca()
plt.text(0.8, 0.9, "(a)", fontsize=20, transform=ax.transAxes)

hist, bin_edges = np.histogram(
    degrees,
    bins=np.logspace(np.log10(degrees.min()), np.log10(degrees.max()), 100),
    density=True,
)
bin_centers = [
    (bin_edges[ix] + bin_edges[ix + 1]) / 2 for ix in range(len(bin_edges) - 1)
]

plt.plot(bin_centers, hist, "o", color=colors[0], markersize=5, label="SP membership")

hist, bin_edges = np.histogram(
    n_starterpacks_created,
    bins=np.logspace(
        np.log10(n_starterpacks_created.min()),
        np.log10(n_starterpacks_created.max()),
        20,
    ),
    density=True,
)
bin_centers = [
    (bin_edges[ix] + bin_edges[ix + 1]) / 2 for ix in range(len(bin_edges) - 1)
]

plt.plot(bin_centers, hist, "^", color=colors[1], markersize=5, label="SPs created")

plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"Degree, $k$")
plt.ylabel(r"$P(k)$")
plt.minorticks_off()
plt.legend(loc="lower left")
plt.minorticks_off()
sns.despine()

plt.subplot(132)
ax = plt.gca()
plt.text(0.8, 0.9, "(b)", fontsize=20, transform=ax.transAxes)

s, p = np.unique(edge_sizes, return_counts=True)
plt.plot(s, p / p.sum(), "o", markersize=5, color=colors[0])
plt.plot([8, 8], [3e-6, 1], "--", label="Min. starter pack size", color=colors[4])
plt.plot([150, 150], [3e-6, 1], "-.", label="Max. starter pack size", color=colors[4])
plt.xscale("log")
plt.yscale("log")
plt.ylim([3e-6, 1])
plt.xlabel(r"Edge size, $s$")
plt.ylabel(r"$P(s)$")
plt.minorticks_off()
sns.despine()

plt.subplot(133)
ax = plt.gca()
plt.text(0.8, 0.9, "(c)", fontsize=20, transform=ax.transAxes)

plt.plot(component_sizes, component_numbers, "o", color=colors[0])
plt.xscale("log")
plt.plot([8, 8], [0, 120], "--", label="Min. SP size", color=colors[4])
plt.plot([150, 150], [0, 120], "-.", label="Max. SP size", color=colors[4])

plt.legend()
plt.ylim([0, 120])
plt.xlabel("Component size")
plt.ylabel("Number")
plt.minorticks_off()
sns.despine()

plt.tight_layout()
plt.savefig("figures/starterpack_stats.pdf", dpi=1000)
plt.show()

In [ ]:
## stats about starter pack sizes
print(f"Min starter pack size is ", min_edge_size)
print(f"Max starter pack size is ", max_edge_size)
print(f"Mean starter pack size is ", round(mean_edge_size, 3))
print(f"Mean starter pack size is ", median_edge_size)
print(f"Mode of the starter pack size is ", mode_edge_size)

# stats about the degree
print(f"Min degree is ", min_degree)
print(f"Max degree is ", max_degree)
print(f"Mean degree is ", round(mean_degree, 3))
print(f"Mean degree is ", median_degree)
print(f"Mode of the degree is ", mode_degree)

In [ ]:
plt.figure(figsize=(11, 4))

plt.subplot(121)
ax = plt.gca()
plt.text(0.8, 0.9, "(a)", fontsize=20, transform=ax.transAxes)
bins = np.arange(
    min(creation_dates), max(creation_dates) + timedelta(days=1), timedelta(days=1)
)

c, t = np.histogram(creation_dates, bins=bins)

plt.plot(t[:-1], c, "o", markersize=5, color=colors[0])

plt.xticks([min(bins), max(bins)])
plt.yticks([0, 7e3, 14e3], ["0", "7000", "14000"])
plt.minorticks_off()
sns.despine()

plt.xlabel(r"Date")
plt.ylabel(r"Number of SPs created")

plt.subplot(122)
ax = plt.gca()
plt.text(0.8, 0.9, "(b)", fontsize=20, transform=ax.transAxes)
d, p = np.unique(account_age_at_creation, return_counts=True)
plt.plot(d, p / p.sum(), "o", markersize=5, color=colors[0])
plt.yscale("log")
plt.xlabel(r"Account age at SP creation (days)")
plt.ylabel(r"Prob.")
plt.minorticks_off()
sns.despine()

plt.tight_layout()
plt.savefig("figures/temporal_starterpack_stats.pdf", dpi=1000)
plt.show()

In [ ]:
df = pd.read_csv("data/s_count.txt", sep="\t")

plt.figure(figsize=(11, 5), dpi=100)

plt.subplot(121)
ax = plt.gca()
plt.text(0.15, 0.15, "(a)", fontsize=20, transform=ax.transAxes)

ymin = 1e-6
ymax = max(df["nodes"] / num_nodes) * 1.3
plt.plot(df["s"], df["nodes"] / num_nodes, "o", color=colors[0], markersize=5)
plt.plot([150, 150], [ymin, ymax], "-.", label="Max. SP size", color=colors[4])

plt.xlabel(r"Edge overlap, $s$")
plt.ylabel("Fraction of SPs")
plt.yscale("log")
plt.xscale("log")
plt.minorticks_off()
sns.despine()

plt.ylim([ymin, ymax])

plt.subplot(122)
ax = plt.gca()
plt.text(0.15, 0.15, "(b)", fontsize=20, transform=ax.transAxes)

max_edges = (num_edges - 1) * num_edges / 2
ymin = 1.5e-11
ymax = max(df["edges"] / max_edges) * 1.3
plt.plot(df["s"], df["edges"] / max_edges, "o", color=colors[1], markersize=5)
plt.plot([150, 150], [ymin, ymax], "-.", label="Max. SP size", color=colors[4])

plt.xlabel(r"Edge overlap, $s$")
plt.ylabel("Line graph density")
plt.yscale("log")
plt.xscale("log")
plt.minorticks_off()
sns.despine()
plt.ylim([ymin, ymax])

plt.tight_layout()
plt.savefig("figures/starterpack_s-line-graph_stats.pdf", dpi=1000)

In [ ]:
cooccurrence_counts = np.loadtxt(
    "data/starterpack_pair_co-occurrence.csv.gz", dtype=int
)
k_core = np.loadtxt("data/starterpack_k-core.csv.gz", dtype=int)
cuts = pd.read_csv("data/starterpack_clustering.csv.gz")

In [ ]:
min_core = min(k_core)
max_core = max(k_core)
num_bins = 100
bins = np.logspace(np.log10(min_core), np.log10(max_core), num=num_bins)

p1, bin_edges1 = np.histogram(k_core, bins=bins, density=True)
x1 = np.sqrt(bin_edges1[:-1] * bin_edges1[1:])


min_count = min(cooccurrence_counts)
max_count = max(cooccurrence_counts)
num_bins = 75
bins = np.logspace(np.log10(min_count), np.log10(max_count), num=num_bins)

p2, bin_edges2 = np.histogram(cooccurrence_counts, bins=bins, density=True)
x2 = np.sqrt(bin_edges2[:-1] * bin_edges2[1:])

sizes = cuts["size"]
fractions = cuts["fraction"]

In [ ]:
plt.figure(figsize=(11, 4))
plt.subplot(131)
ax = plt.gca()
plt.text(0.8, 0.9, "(a)", fontsize=20, transform=ax.transAxes)

plt.loglog(x1, p1, "o", markersize=5, color=colors[0])

plt.xlabel(r"Coreness, $k$")
plt.ylabel(r"$P(k)$")
plt.minorticks_off()
sns.despine()

plt.subplot(132)
ax = plt.gca()
plt.text(0.8, 0.9, "(b)", fontsize=20, transform=ax.transAxes)
plt.loglog(x2, p2, "o", markersize=5, color=colors[0])
plt.xlabel("# of co-occurring pairs")
plt.ylabel("Prob.")
plt.minorticks_off()
sns.despine()

plt.subplot(133)
ax = plt.gca()
plt.text(0.8, 0.9, "(c)", fontsize=20, transform=ax.transAxes)
plt.plot(sizes, fractions, "o", markersize=5, color=colors[0]), colors
plt.xscale("log")
plt.xlabel(r"Hyperedge size, $s$")
plt.ylabel("Fraction of uncut edges")
plt.plot([8, 8], [0, 1], "--", color=colors[4], label="Min. SP size")
plt.plot([150, 150], [0, 1], "-.", color=colors[4], label="Max. SP size")
plt.legend()
plt.minorticks_off()
sns.despine()
plt.tight_layout()
plt.savefig("figures/starterpack_misc_stats.pdf")